In [ ]:
%load_ext autoreload
%autoreload 2

In [17]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false
}

<IPython.core.display.Javascript object>

In [18]:
from CoreConfig.CoreConfig import CoreConfig
from DBUtility.DBUtility import DBSQLiteConnection, CreateEventsTableIfNotExist, CreateNPSTableIfNotExist
from DBUtility.DBUtility import FetchEventsRecordWithinSamePerson, FetchEventsLatestRecordWithSamePerson, FetchEventsRecordAll, FetchNPSRecordAll
from DBUtility.DBUtility import InsertEventRecord, InsertNPSRecord
from pandas import DataFrame
import datetime


import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
from IPython.display import display_pretty
import pandas as pd
from ipywidgets import Layout, Button, Box
from StylingUtility.StylingUtility import CustomStyling

# CoreConfig.initialize_core_config()
# DBSQLiteConnection().configure_connection_information(**CoreConfig().get_sqlite3_db_connection_info())
# CreateEventsTableIfNotExist().execute()


### NPS Form

In [19]:
# get configuration details
CoreConfig.initialize_core_config()

# establish database connection
DBSQLiteConnection().configure_connection_information(**CoreConfig().get_sqlite3_db_connection_info())

# create table if not exist
CreateNPSTableIfNotExist().execute()

Initialize nab NPS table failed. Please check your connection
table nab_mobile_nps already exists


In [20]:
title_nps = widgets.HTML("<center><H3>Mobile NPS Survey</H3><center>")

formtitle_nps = widgets.HTML("<center><H3>Digital Channel - Internal Survey</H3><center>")

df_out_nps = widgets.Output(layout={'border': '1px solid black', 'aligh_items' : 'center'})

with df_out_nps:
    df = DataFrame(FetchNPSRecordAll().execute(), columns=['Date', 'Channel', 'Score', 'Thoughts', 'Comments'])
    display(df)

date = widgets.DatePicker(
        value = datetime.datetime.now().date(),
        description='Pick a Date',
        layout=CustomStyling.box_layout,
        style = CustomStyling.style,
        disable=False)
    
channel = widgets.Dropdown(options= {'NAB Connect': 'NAB Connect',
                                     'NAB Connect Mobile': 'NAB Connect Mobile',
                                    'NAB Internet Banking':'NAB Internet Banking',
                                    'NAB Mobile App': 'NAB Mobile App',
                                    'NAB.com.au': 'NAB.com.au'},
                           value = 'NAB Mobile App',
                           description='Select channel:',
                           layout=CustomStyling.box_layout,
                           style=CustomStyling.style)

q1 = widgets.IntSlider(
        value=7,
        min=0,
        max=10,
        step=1,
        description='How likely are you to recommend working in the Mobile Service to others?:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        layout=CustomStyling.box_layout,
        style=CustomStyling.style,
        readout_format='d')
    
q2 = widgets.Text(
      value='',
      placeholder='Type Here',
      description='Share your thoughts on how can we improve the Mobile Service?',
      layout=CustomStyling.box_layout,
       style=CustomStyling.style,
      disable=False)

q3 = widgets.Text(
      value='',
      placeholder='Type Here',
      description='Any additional comments?',
      layout=CustomStyling.box_layout,
       style=CustomStyling.style,
      disable=False)

button_nps = widgets.Button(
           description='Submit the form',
           disable=False,
           button_style ='', # success
           tooltip='Click me',
           icon='check'
           )

def on_submit_form_button_clicked_nps(b):
    outputs = { 'date' : [date.value],
                'channel' : [channel.value],
                'q1' : [q1.value],
                'q2' : [q2.value],
                'q3' : [q3.value]
               }
    InsertNPSRecord(date.value, channel.value, q1.value, q2.value, q3.value).execute()
    df = DataFrame(FetchNPSRecordAll().execute(), columns=['Date', 'Channel', 'Score', 'Thoughts', 'Comments'])
    df_out_nps.clear_output()
    with df_out_nps:
        display(df)
        
button_nps.on_click(on_submit_form_button_clicked_nps)

In [21]:
# user interface organised using VBox (vertical)
ui_nps = widgets.VBox(children=[title_nps,
                            df_out_nps,
                            formtitle_nps,
                            date,
                            channel,
                            q1,
                            q2,
                            q3,
                            button_nps], layout=CustomStyling.items_layout)

# live display of values, for debugging purposes
def form_nps(date, channel, q1, q2, q3):
#     display(date, channel, q1, q2, q3)
    return date, channel, q1, q2, q3

out_nps = interactive_output(form_nps, {'date': date,
                                'channel': channel,
                                'q1' : q1,
                                'q2':q2,
                                'q3':q3
                               })

display(ui_nps, out_nps)

A Jupyter Widget

A Jupyter Widget

In [ ]:
out_nps.outputs